<a href="https://colab.research.google.com/github/NicoEssi/Data_Science_Portfolio/blob/master/Stackoverflow_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup

In [0]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Downloading the Stack Overflow Survey Results for 2019
!wget --no-check-certificate \
    "https://drive.google.com/uc?authuser=0&id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download" \
    -O "/tmp/soi_2019.zip"
zip_ref = zipfile.ZipFile("/tmp/soi_2019.zip", 'r')
zip_ref.extractall("/tmp/soi_2019")
zip_ref.close()

--2019-09-10 21:31:19--  https://drive.google.com/uc?authuser=0&id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download
Resolving drive.google.com (drive.google.com)... 172.217.214.113, 172.217.214.139, 172.217.214.102, ...
Connecting to drive.google.com (drive.google.com)|172.217.214.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/uc?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download [following]
--2019-09-10 21:31:19--  https://drive.google.com/uc?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/c6ptpdau77qgve6fe7he9cpip1n5c473/1568145600000/06716978924947585995/*/1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV?e=download [following]
--2019-09-10 21:31:22--  https://doc-14-4c-docs.googleusercontent.com/docs/secu

# 1. Business- & Data Understanding

In [0]:
data = pd.read_csv("/tmp/soi_2019/survey_results_public.csv")
schema = pd.read_csv("/tmp/soi_2019/survey_results_schema.csv")

In [0]:
#for i in range(len(schema)):
#    print(schema.iloc[i].Column + " : " + schema.iloc[i].QuestionText)

In [5]:
print("US Count: " + str(np.sum(data.Country == "United States")))
print("UK Count: " + str(np.sum(data.Country == "United Kingdom")))
print("EU Count: " + str(np.sum(data.Country == "France")
                        + np.sum(data.Country == "Germany")
                        + np.sum(data.Country == "Sweden")
                        + np.sum(data.Country == "Denmark")
                        + np.sum(data.Country == "Finland")
                        + np.sum(data.Country == "Ireland")
                        + np.sum(data.Country == "Netherlands")
                        + np.sum(data.Country == "Austria")
                        + np.sum(data.Country == "Belgium")
                        + np.sum(data.Country == "Switzerland")))

US Count: 20949
UK Count: 5737
EU Count: 15591


In [33]:
print("Before: " + str(len(data.CompTotal)))

data = data[pd.notnull(data['CompFreq'])]
data = data[pd.notnull(data['CompTotal'])].reset_index(drop = True)

print("After: " + str(len(data.CompTotal)))

Before: 88883
After: 55827


In [7]:
print("US Count: " + str(np.sum(data.Country == "United States")))
print("UK Count: " + str(np.sum(data.Country == "United Kingdom")))
print("EU Count: " + str(np.sum(data.Country == "France")
                        + np.sum(data.Country == "Germany")
                        + np.sum(data.Country == "Sweden")
                        + np.sum(data.Country == "Denmark")
                        + np.sum(data.Country == "Finland")
                        + np.sum(data.Country == "Ireland")
                        + np.sum(data.Country == "Netherlands")
                        + np.sum(data.Country == "Austria")
                        + np.sum(data.Country == "Belgium")
                        + np.sum(data.Country == "Switzerland")
                        + np.sum(data.Country == "Norway")))

US Count: 14981
UK Count: 4036
EU Count: 10744


## We are interested in identifying the profiles of the top earners in each region; United States, United Kingdom, and Europe. Thereafter, we'll build a model to predict salary.

### Why do we conduct our inquiry on two countries, yet one is of a larger region consisting of numerous countries (Europe)?
Professionals residing within the European Union have liberties granted to them by the Schengen agreement, which enables free movement of labor force and thus professionals can freely work wherever they wish.

### Why are Norway and Switzerland included in the European dataset?
While they are not part of the European Union, they have signed agreements in association with the Schengen agreement.

### Why is the United Kingdom not included in the European dataset despite having signed an agreement in association with the Schengen agreement?
Brexit. Despite the fact that the data has been gathered prior to the secession deadline, it would still be interesting to inquire on the UK data separately in case it would be of interest to compare changes in 2020 and onwards. And as of writing this, nothing has been officially said regarding future signed agreement in association with the Schengen agreement for UK.

### Why are the countries in APT / RCEP not included?
Unfortunately the data gathered for China, South Korea, and Japan are too small to make reliable inferences - and there are currently significant restrictions in freedom of movement for professionals between these countries, making it unreasonable to conduct an inquiry on these countries as a collective.

# 2. Data Preparation

In [36]:
#del data['Respondent']

data = data[data["MainBranch"] == 'I am a developer by profession']
data = data[data["LastHireDate"] != "I've never had a job"]

len(data)

50397

In [41]:
data.LanguageWorkedWith.unique()

array(['C;C++;C#;Python;SQL',
       'Bash/Shell/PowerShell;C#;HTML/CSS;JavaScript;Python;Ruby;Rust;SQL;TypeScript;WebAssembly;Other(s):',
       'C#;Go;JavaScript;Python;R;SQL', ...,
       'Bash/Shell/PowerShell;C#;HTML/CSS;Java;Python;R;Scala',
       'Bash/Shell/PowerShell;C;Clojure;HTML/CSS;Java;Scala;Swift;Other(s):',
       'Go;HTML/CSS;Java;JavaScript;Other(s):'], dtype=object)

In [0]:
relevant = ["Hobbyist", "OpenSourcer", "Employment", "Country",
"Student", "EdLevel", "UndergradMajor", "EduOther", "OrgSize",
"DevType", "YearsCode", "YearsCodePro", "JobSeek", "LastHireDate",
"FizzBuzz", "CompTotal", "CompFreq", "CurrencySymbol",
"LanguageWorkedWith", "DatabaseWorkedWith", "PlatformWorkedWith", "WebFrameWorkedWith", "MiscTechWorkedWith",
"Age", "Gender", "Ethnicity", "Dependents"]

data_us = data[data["Country"] == "United States"]
data_us = data_us[relevant]

data_uk = data[data["Country"] == "United Kingdom"]
data_uk = data_uk[relevant]

data_eu = data[data["Country"].isin(["France", "Germany", "Sweden",
                                     "Denmark", "Finland", "Norway",
                                     "Ireland", "Netherlands", "Austria",
                                     "Belgium", "Switzerland"])]
data_eu = data_eu[relevant]

data_test = data[data["Country"] == "Canada"]
data_test = data_test[relevant]

In [0]:
# currency rates as of 09/09/2019
"""
EUR = 1.10
CHF = 1.00
SEK = 0.10
NOK = 0.11
DKK = 0.15
USD = 1.00
GBP = 1.23
INR = 0.014
NZD = 0.64
AUD = 0.69
CAD = 0.76
"""

currencies = {"EUR" : 1.10
             ,"CHF" : 1.00
             ,"SEK" : 0.10
             ,"NOK" : 0.11
             ,"DKK" : 0.15
             ,"USD" : 1.00
             ,"GBP" : 1.23
             ,"INR" : 0.014
             ,"NZD" : 0.64
             ,"AUD" : 0.69
             ,"CAD" : 0.76}

def currency_usd(data):
    if data in (currencies):
        return currencies[data]
    else:
        return 0

def currency_annualize(data):
    if data == 'Weekly':
        return 52
    elif data == 'Monthly':
        return 12
    elif data == 'Yearly':
        return 1

In [8]:
data_us["CompTotal"] *= data_us["CompFreq"].apply(currency_annualize)
data_us["CompTotal"] *= data_us["CurrencySymbol"].apply(currency_usd)
data_us = data_us[data_us.CompTotal > 100]

data_uk["CompTotal"] *= data_uk["CompFreq"].apply(currency_annualize)
data_uk["CompTotal"] *= data_uk["CurrencySymbol"].apply(currency_usd)
data_uk = data_uk[data_uk.CompTotal > 100]

data_eu["CompTotal"] *= data_eu["CompFreq"].apply(currency_annualize)
data_eu["CompTotal"] *= data_eu["CurrencySymbol"].apply(currency_usd)
data_eu = data_eu[data_eu.CompTotal > 100]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [44]:
languages_temp = []
for i in range( len(data_eu.LanguageWorkedWith) ):
    languages_temp.append([parsed.strip() for parsed in str(data_eu.LanguageWorkedWith.iloc[i]).split(';')])

languages = []
for i in languages_temp:
    for j in i:
        if j not in languages:
            languages.append(j)
            
languages

['C++',
 'nan',
 'Java',
 'Kotlin',
 'Python',
 'Bash/Shell/PowerShell',
 'C#',
 'HTML/CSS',
 'JavaScript',
 'SQL',
 'TypeScript',
 'WebAssembly',
 'Go',
 'PHP',
 'Ruby',
 'VBA',
 'Clojure',
 'R',
 'Objective-C',
 'Assembly',
 'Swift',
 'Dart',
 'Scala',
 'C',
 'Other(s):',
 'Rust',
 'Erlang',
 'Elixir',
 'F#']

In [163]:
# Handling DevType

devtypes_temp = []
for i in range( len(data_eu.DevType) ):
    devtypes_temp.append([parsed.strip() for parsed in str(data_eu.DevType.iloc[i]).split(';')])

devtypes = []
for i in devtypes_temp:
    for j in i:
        if j not in devtypes:
            devtypes.append(j)
            
devtypes

['Developer, desktop or enterprise applications',
 'DevOps specialist',
 'Engineer, site reliability',
 'Developer, mobile',
 'Student',
 'Developer, back-end',
 'Developer, full-stack',
 'Developer, front-end',
 'System administrator',
 'nan',
 'Data or business analyst',
 'Data scientist or machine learning specialist',
 'Database administrator',
 'Designer',
 'Educator',
 'Marketing or sales professional',
 'Developer, QA or test',
 'Engineer, data',
 'Engineering manager',
 'Product manager',
 'Developer, game or graphics',
 'Academic researcher',
 'Scientist',
 'Developer, embedded applications or devices',
 'Senior executive/VP']

In [9]:
relevant = ["MainBranch", "Hobbyist", "OpenSourcer", "Employment", "Country",
"Student", "EdLevel", "UndergradMajor", "EduOther", "OrgSize",
"DevType", "YearsCode", "YearsCodePro", "JobSeek", "LastHireDate",
"FizzBuzz", "JobFactors", "CompTotal", "ConvertedComp", "WorkWeekHrs",
"LanguageWorkedWith", "DatabaseWorkedWith", "PlatformWorkedWith", "WebFrameWorkedWith", "MiscTechWorkedWith",
"Age", "Gender", "Ethnicity", "Dependents"]

data = data[relevant]

data

,MainBranch,Hobbyist,OpenSourcer,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,YearsCodePro,JobSeek,LastHireDate,FizzBuzz,JobFactors,CompTotal,ConvertedComp,WorkWeekHrs,LanguageWorkedWith,DatabaseWorkedWith,PlatformWorkedWith,WebFrameWorkedWith,MiscTechWorkedWith,Age,Gender,Ethnicity,Dependents
0,"I am not primarily a developer, but I write co...",Yes,Never,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,"Taught yourself a new language, framework, or ...",100 to 499 employees,"Designer;Developer, back-end;Developer, front-...",3,1,"I’m not actively looking, but I am open to new...",1-2 years ago,No,"Languages, frameworks, and other technologies ...",23000.0,8820.0,40.00,HTML/CSS,PostgreSQL,NaN,NaN,NaN,28.0,Man,NaN,Yes
1,I am a developer by profession,No,Never,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,100 to 499 employees,"Developer, full-stack",3,Less than 1 year,I am not interested in new job opportunities,Less than a year ago,No,"Languages, frameworks, and other technologies ...",61000.0,61000.0,80.00,C;C++;C#;Python;SQL,MySQL;SQLite,Linux;Windows,NaN,.NET,22.0,Man,White or of European descent,No
2,"I am not primarily a developer, but I write co...",Yes,Never,Employed full-time,Canada,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,Taken an online course in programming or softw...,NaN,Data or business analyst;Data scientist or mac...,13,3,I am not interested in new job opportunities,1-2 years ago,No,Financial performance or funding status of the...,40000.0,366420.0,15.00,Java;R;SQL,MongoDB;PostgreSQL,Android;Google Cloud Platform;Linux;Windows,NaN,Hadoop,28.0,Man,East Asian,No
3,I am a developer by profession,Yes,Once a month or more often,Employed full-time,New Zealand,No,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,10 to 19 employees,"Database administrator;Developer, back-end;Dev...",12,4,"I’m not actively looking, but I am open to new...",Less than a year ago,Yes,Financial performance or funding status of the...,138000.0,95179.0,32.00,Bash/Shell/PowerShell;C#;HTML/CSS;JavaScript;P...,DynamoDB;PostgreSQL;SQLite,AWS;Docker;Heroku;Linux;MacOS;Slack,Express;Ruby on Rails;Other(s):,Node.js;Unity 3D,23.0,Man,White or of European descent,No
4,I am a developer by profession,Yes,Once a month or more often,Employed full-time,India,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",NaN,NaN,"10,000 or more employees",Data or business analyst;Data scientist or mac...,12,10,"I’m not actively looking, but I am open to new...",3-4 years ago,No,"Languages, frameworks, and other technologies ...",950000.0,13293.0,70.00,C#;Go;JavaScript;Python;R;SQL,Elasticsearch;MongoDB;Microsoft SQL Server;MyS...,Linux;Windows,Angular/Angular.js;ASP.NET;Django;Express;Flas...,.NET;Node.js;Pandas;Torch/PyTorch,NaN,NaN,NaN,Yes
5,I am a developer by profession,Yes,Less than once a month but more than once per ...,Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,10 to 19 employees,Data or business analyst;Database administrato...,17,8,I am not interested in new job opportunities,3-4 years ago,Yes,"Languages, frameworks, and other technologies ...",90000.0,90000.0,40.00,Bash/Shell/PowerShell;HTML/CSS;JavaScript;PHP;...,Couchbase;DynamoDB;Firebase;MySQL,Android;AWS;Docker;IBM Cloud or Watson;iOS;Lin...,Angular/Angular.js;ASP.NET;Express;jQuery;Vue.js,Node.js;Xamarin,28.0,Man,White or of European descent,Yes
6,I am a developer by profession,Yes,Less than once per year,Employed full-time,Germany,No,"Other doctoral degree (Ph.D, Ed.D., etc.)","Computer science, computer engi

In [115]:
for i in range( len(schema) ):
    print(schema.Column[i])

Respondent : Randomized respondent ID number (not in order of survey response time)
MainBranch : Which of the following options best describes you today? Here, by "developer" we mean "someone who writes code."
Hobbyist : Do you code as a hobby?
OpenSourcer : How often do you contribute to open source?
OpenSource : How do you feel about the quality of open source software (OSS)?
Employment : Which of the following best describes your current employment status?
Country : In which country do you currently reside?
Student : Are you currently enrolled in a formal, degree-granting college or university program?
EdLevel : Which of the following best describes the highest level of formal education that you’ve completed?
UndergradMajor : What was your main or most important field of study?
EduOther : Which of the following types of non-degree education have you used or participated in? Please select all that apply.
OrgSize : Approximately how many people are employed by the company or organizat